In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:1600px !important; }</style>"))

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
import datetime

In [ ]:
%matplotlib inline
if "loudness" in plt.style.available:
    plt.style.use("loudness")
matplotlib.rcParams["font.size"] = 16.0
matplotlib.rcParams["figure.figsize"] = (15.0, 8.0)

In [ ]:
from fastai.vision import *

In [ ]:
path = "/home/andras/Dokumente/Kaggle/histopathologic-cancer-detection/all"

In [ ]:
# TODOS:
## use seed to get same results

In [ ]:
# data = ImageDataBunch.from_csv(path, ds_tfms=get_transforms(), size=96, test=path).normalize(imagenet_stats)
data = ImageDataBunch.from_csv(path, folder="train", csv_labels="train_labels.csv", ds_tfms=get_transforms(), size=96, test="test", suffix=".tif")

In [ ]:
data

In [ ]:
data.show_batch(rows=4)

## Resnet 18

In [ ]:
learn = create_cnn(data, models.resnet18, metrics=accuracy)
learn.fit_one_cycle(1)

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(6, slice(1e-5,3e-4), pct_start=0.05)

In [ ]:
accuracy(*learn.TTA())

In [ ]:
assert False

## Resnet 34

In [ ]:
learn = create_cnn(data, models.resnet34, metrics=accuracy)
learn.fit_one_cycle(1)

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(6, slice(1e-5,3e-4), pct_start=0.05)

In [ ]:
accuracy(*learn.TTA())

In [ ]:
assert False

## Resnet 50

In [ ]:
learn = create_cnn(data, models.resnet50, metrics=accuracy)
learn.fit_one_cycle(1)

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(6, slice(1e-5,3e-4), pct_start=0.05)

In [ ]:
accuracy(*learn.TTA())

In [ ]:
assert False

## Resnet 101

In [ ]:
learn = create_cnn(data, models.resnet101, metrics=accuracy)
learn.fit_one_cycle(1)

In [ ]:
learn.recorder.plot()

In [ ]:
learn.recorder.plot_losses()

In [ ]:
learn.unfreeze()
# learn.fit_one_cycle(6, slice(1e-5,3e-4), pct_start=0.05)
learn.fit_one_cycle(8, slice(1e-5,3e-4))

In [ ]:
accuracy(*learn.TTA())

In [ ]:
learn.recorder.plot()

In [ ]:
learn.recorder.plot_losses()

In [ ]:
learn.save(datetime.datetime.now().strftime("%Y%m%d_%H%M") + "_resnet101.model")

In [ ]:
assert False

## Resnet 152

In [ ]:
learn = create_cnn(data, models.resnet152, metrics=accuracy)
learn.fit_one_cycle(1)

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(6, slice(1e-5,3e-4), pct_start=0.05)

In [ ]:
accuracy(*learn.TTA())

In [ ]:
assert False

## Densenet 121

In [ ]:
from torchvision.models import *

In [ ]:
learn = create_cnn(data, densenet121, metrics=accuracy)
learn.fit_one_cycle(1)

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(6, slice(1e-5,3e-4), pct_start=0.05)

In [ ]:
accuracy(*learn.TTA())

In [ ]:
learn.save(datetime.datetime.now().strftime("%Y%m%d_%H%M") + "_densenet121.model")

In [ ]:
assert False

# Checking the KPIs

In [ ]:
from sklearn.metrics import roc_auc_score

def auc_score(y_pred, y_true):
    return roc_auc_score(to_np(y_true),to_np(y_pred)[:,1])

In [ ]:
preds_valid, y_valid = learn.TTA(ds_type=DatasetType.Valid)

In [ ]:
print("Accuracy: %g" % accuracy(preds_valid, y_valid))
print("AUROCC: %g" % auc_score(preds_valid, y_valid))

## Prepare the submission file

In [ ]:
preds_test, y_test = learn.TTA(ds_type=DatasetType.Test)
# preds_test, y_test = learn.get_preds(ds_type=DatasetType.Test)

In [ ]:
# create submission file
submission_id = [str(x).replace(path + "/test/", "").replace(".tif", "") for x in data.test_ds.items]

In [ ]:
# use probabilities
submission_label = to_np(preds_test[:,1])
# use discrete values
# submission_label = to_np(preds_test.argmax(dim=1))

In [ ]:
df_submission = pd.DataFrame(data=submission_label, index=submission_id, columns=["label"])

In [ ]:
df_submission.index.name = "id"

In [ ]:
df_submission.head(10)

In [ ]:
len(df_submission)

In [ ]:
df_submission.to_csv(datetime.datetime.now().strftime("%Y%m%d_%H%M") + " submission.csv")

# Explore missclassifications

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_top_losses(9, figsize=(12,12))

In [ ]:
interp.plot_confusion_matrix()